In [ ]:
def loadFile(fileName):
    array = np.load(fileName)
    
    return array

In [ ]:
def showImage(inTainSet, atPosition):
    img = inTainSet[atPosition,:,:,:]/255.0
    plt.imshow(img)

In [ ]:
def normalize_image(trainingSet):
    size = trainingSet.shape[0]*trainingSet.shape[1]*trainingSet.shape[2]*trainingSet.shape[3] #total number of pixel in the training set
    accum = 0 #variable to sum all the values
    
    for im in range(0,trainingSet.shape[0]): #pass throught all images
        for row in range(0,trainingSet.shape[1]): #pass throught all rows in the current image
            for col in range(0,trainingSet.shape[2]):  #pass throught all column in the current row of the current image 
                for channel in range(0,trainingSet.shape[3]): #pass throught all channel in the current column of the current row of the current image
                    accum += trainingSet[im,row,col,channel]
                    
    mean = accum / (size) #calculate the mean pixel value of the training set
    
    for im in range(0,trainingSet.shape[0]):
        for row in range(0,trainingSet.shape[1]): 
            for col in range(0,trainingSet.shape[2]):   
                for channel in range(0,trainingSet.shape[3]):
                    trainingSet[im,row,col,channel] = (trainingSet[im,row,col,channel] - mean)/mean # substract the mean value to all pixel of the training set

In [ ]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [ ]:
def random_mini_batches(X, Y, mini_batch_size):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (number of examples, img width, image height, number of channel)
    Y -- true "label" vector (containing int number of the class value), of shape (number of examples, 1)
    mini_batch_size - size of the mini-batches, integer
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation, :, :, :]
    shuffled_Y = Y[permutation, :].reshape((m, Y.shape[1]))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size, :, :, :]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size, :]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m, :, :, :]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m, :]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches


In [7]:
def create_convolutional_layer(input,
               num_input_channels, 
               conv_filter_size,        
               num_filters,
               padding,
               strides,
               layer_number, use_bias = True):  
    """
    TF's conv2d function calculates convolutions in batches and uses a slightly different format. 
    For an input it is [batch, in_height, in_width, in_channels] 
    For the filter it is [filter_height, filter_width, in_channels, out_channels]
    padding: 'SAME' or 'VALID'
    strides: [data, height, width, channel], usually [1, 2, 2, 1] to reduce data size per 2
    """
    
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(name='Wconv_'+layer_number, shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(name='Bconv_'+layer_number, size=num_filters)
 
    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=strides,
                     padding=padding)
 
    if use_bias:
        layer += biases
 
    return layer

In [6]:
import tensorflow as tf

def create_weights(shape, name, initializer= tf.contrib.layers.xavier_initializer()):
    return tf.get_variable(name=name, shape=shape, initializer= initializer)
 
def create_biases(size, name, initializer= tf.zeros_initializer()):
    return tf.get_variable(name=name, shape=[size], initializer= initializer)

In [3]:
def create_placeholders(img_size, num_channels, num_classes):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_x -- scalar, size of an image vector (num_px * num_px = 64 * 64 * 3 = 12288)
    n_y -- scalar, number of classes (from 0 to 199, so -> 200)
    
    Returns:
    X -- placeholder for the data input, of shape [n_x, None] and dtype "float"
    Y -- placeholder for the input labels, of shape [n_y, None] and dtype "float"
    
    Tips:
    - Use None because it let's us be flexible on the number of examples you will for the placeholders.
      In fact, the number of examples during test/train is different.
    """
    
    X = tf.placeholder(name='X', dtype=tf.float32, shape=[None, img_size,img_size,num_channels])
    Y = tf.placeholder(name='Y', dtype=tf.int32, shape=[None, num_classes])

    return X, Y